## Part 0: Check Model Groups in my AWS

In [2]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all model package groups
response = sagemaker_client.list_model_package_groups()

# Print the list of model package groups
for group in response['ModelPackageGroupSummaryList']:
    print(f"Model Group Name: {group['ModelPackageGroupName']}, Created On: {group['CreationTime']}")

## Part 1: Set Up Model Group

- The Model Group will contain a group of versioned models.  
- Every time we update the model algorithm, input data, features, model hyperparameters, etc. we will add a new model package to this group.

In [3]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Define the Model Package Group parameters
model_package_group_name = "xgboost-diabetes-detection"
model_package_group_description = "Model group for versioned models of XGBoost for diabetes detection."

# Create the Model Package Group
response = sagemaker_client.create_model_package_group(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageGroupDescription=model_package_group_description
)

print("Model Package Group ARN:", response['ModelPackageGroupArn'])


Model Package Group ARN: arn:aws:sagemaker:us-east-1:333907264027:model-package-group/xgboost-diabetes-detection


In [4]:
# Describe the Model Package Group
describe_response = sagemaker_client.describe_model_package_group(
    ModelPackageGroupName=model_package_group_name
)

print(describe_response)

{'ModelPackageGroupName': 'xgboost-diabetes-detection', 'ModelPackageGroupArn': 'arn:aws:sagemaker:us-east-1:333907264027:model-package-group/xgboost-diabetes-detection', 'ModelPackageGroupDescription': 'Model group for versioned models of XGBoost for diabetes detection.', 'CreationTime': datetime.datetime(2024, 6, 4, 6, 6, 38, 656000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:333907264027:user-profile/d-zegxltkmyanc/maxrep', 'UserProfileName': 'maxrep', 'DomainId': 'd-zegxltkmyanc', 'IamIdentity': {'Arn': 'arn:aws:sts::333907264027:assumed-role/LabRole/SageMaker', 'PrincipalId': 'AROAU3PTMQINWIR2EC7KE:SageMaker'}}, 'ModelPackageGroupStatus': 'Completed', 'ResponseMetadata': {'RequestId': '2d4debe9-6936-4554-b778-872789d6b862', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2d4debe9-6936-4554-b778-872789d6b862', 'content-type': 'application/x-amz-json-1.1', 'content-length': '631', 'date': 'Tue, 04 Jun 2024 06:06:42 GMT'}, 'RetryAttemp

## Part 2: Set Up Model Package

The Model Package will contain specific details about our current model:  
- Our Model Package should document model deployment information (instance image, model data source i.e. our binary artifact, data source, any pre-processor or post-processor scripts, etc.).  
- After we learn more about Model monitoring, we can also include model quality, model data quality, model bias and model explainability reports here too!

### - Default Bucket

In [5]:
import sagemaker

# Initialize the SageMaker session
sess = sagemaker.Session()

# Get the default S3 bucket
bucket = sess.default_bucket()
print("Default S3 bucket:", bucket)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Default S3 bucket: sagemaker-us-east-1-333907264027


### - Model Package

In [15]:
# Verify the S3 objects in the specified bucket and prefix
import boto3

s3_client = boto3.client('s3')
response = s3_client.list_objects_v2(Bucket=bucket, Prefix='DEMO-diabetes-prediction-xgboost/output/')

for obj in response.get('Contents', []):
    print(obj['Key'])

DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/debug-output/training_job_end.ts
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/framework/training_job_end.ts
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/system/incremental/2024060401/1717463580.algo-1.json
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/system/incremental/2024060401/1717463640.algo-1.json
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/system/incremental/2024060401/1717463700.algo-1.json
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/system/training_job_end.ts
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-39-00/xgb-2024-06-04-01-39-00/debug-output/training_job_end.ts
DEMO-diabetes-predicti

In [6]:
import boto3
import sagemaker

# Initialize the SageMaker session
sess = sagemaker.Session()

# Get the default S3 bucket
bucket = sess.default_bucket()
print("Default S3 bucket:", bucket)

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Define the model package parameters
model_package_group_name = "xgboost-diabetes-detection"
model_package_name = "xgboost-diabetes-detection-v1"
model_data_url = "s3://{}/DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-02-10-33/xgb-2024-06-04-02-10-33/output/model.tar.gz".format(bucket)
container_image_uri = sagemaker.image_uris.retrieve("xgboost", region=boto3.Session().region_name, version="1.7-1")

# Create the Model Package
create_model_package_response = sagemaker_client.create_model_package(
    ModelPackageGroupName=model_package_group_name,
    ModelPackageDescription="XGBoost model package for diabetes detection",
    ModelApprovalStatus="Approved",
    InferenceSpecification={
        "Containers": [
            {
                "Image": container_image_uri,
                "ModelDataUrl": model_data_url,
                "Environment": {}
            }
        ],
        "SupportedTransformInstanceTypes": [
            "ml.m4.xlarge"
        ],
        "SupportedRealtimeInferenceInstanceTypes": [
            "ml.m4.xlarge"
        ],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"]
    }
)

print("Model Package ARN:", create_model_package_response['ModelPackageArn'])

# Describe the Model Package
describe_model_package_response = sagemaker_client.describe_model_package(
    ModelPackageName=create_model_package_response['ModelPackageArn']
)

print(describe_model_package_response)


Default S3 bucket: sagemaker-us-east-1-333907264027
Model Package ARN: arn:aws:sagemaker:us-east-1:333907264027:model-package/xgboost-diabetes-detection/1
{'ModelPackageGroupName': 'xgboost-diabetes-detection', 'ModelPackageVersion': 1, 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:333907264027:model-package/xgboost-diabetes-detection/1', 'ModelPackageDescription': 'XGBoost model package for diabetes detection', 'CreationTime': datetime.datetime(2024, 6, 4, 6, 6, 53, 122000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1', 'ImageDigest': 'sha256:cf81520a3b695293022793e292cf8bc3732b79231a6ebe1fb308086f6163a875', 'ModelDataUrl': 's3://sagemaker-us-east-1-333907264027/DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-02-10-33/xgb-2024-06-04-02-10-33/output/model.tar.gz', 'Environment': {}}], 'SupportedTransformInstanceTypes': ['ml.m4.xlarge'], 'SupportedRealtimeInferenceInstanceTypes': ['m

### - Delete a Model Group (including its Model Packages)

In [7]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Define the Model Group name that you want to DELETE !!!
model_group_name = "xgboost-diabetes-detection"

# List all Model Packages in the Model Group
paginator = sagemaker_client.get_paginator('list_model_packages')
response_iterator = paginator.paginate(
    ModelPackageGroupName=model_group_name
)

# Collect all model package names
model_package_names = []
for page in response_iterator:
    for model_package_summary in page['ModelPackageSummaryList']:
        model_package_names.append(model_package_summary['ModelPackageArn'])

# Delete each Model Package in the Model Group
for model_package_arn in model_package_names:
    print(f"Deleting Model Package: {model_package_arn}")
    sagemaker_client.delete_model_package(
        ModelPackageName=model_package_arn
    )

# Delete the Model Group
sagemaker_client.delete_model_package_group(
    ModelPackageGroupName=model_group_name
)

print(f"Deleted Model Group: {model_group_name}")

Deleting Model Package: arn:aws:sagemaker:us-east-1:333907264027:model-package/xgboost-diabetes-detection/1
Deleted Model Group: xgboost-diabetes-detection


## Part 3: Write the Model Card

Review the API documentation AND the Model Card JSON Schema and fill out what you can.  
- Again, it’s important to document what is useful to you and your team, but keep documentation brief so it is still readable.  
- What we really want to get out of the Model Card is a snapshot of the model and its purpose so that we remember why and how a model was built.  
- Additionally, we want to document every experiment that we tried to improve our model’s performance so that we don’t keep trying the same thing over and over and waste engineering effort during model development and optimization.

### - List ALL 'Existing' Model Cards

In [8]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all Model Cards
list_model_cards_response = sagemaker_client.list_model_cards()

# Print the list of Model Cards
for model_card in list_model_cards_response.get('ModelCardSummaries', []):
    print(f"Model Card Name: {model_card['ModelCardName']}, Creation Time: {model_card['CreationTime']}")

# If there are additional attributes, print them
for model_card in list_model_cards_response.get('ModelCardSummaries', []):
    for key, value in model_card.items():
        print(f"{key}: {value}")
    print("----")
if len(list_model_cards_response.get('ModelCardSummaries', [])) == 0:
    print('No Model Cards Exist')

No Model Cards Exist


In [14]:
import boto3

# Initialize the S3 client
s3_client = boto3.client('s3')

# Define your bucket name
bucket_name = 'sagemaker-us-east-1-333907264027'

# List objects in the bucket
response = s3_client.list_objects_v2(Bucket=bucket_name)

# Print the keys of the objects in the bucket
for obj in response.get('Contents', []):
    print(obj['Key'])

DEMO-diabetes-prediction-xgboost/batch/batch_data.csv
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/debug-output/training_job_end.ts
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/framework/training_job_end.ts
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/system/incremental/2024060401/1717463580.algo-1.json
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/system/incremental/2024060401/1717463640.algo-1.json
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/system/incremental/2024060401/1717463700.algo-1.json
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-13-05/xgb-2024-06-04-01-13-05/profiler-output/system/training_job_end.ts
DEMO-diabetes-prediction-xgboost/output/xgb-2024-06-04-01-39-00/xgb-2024-06-04-01-39-00/d

### - Create a Basic Model Card

In [9]:
import boto3
import json

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Define the Model Card parameters
model_card_name = "xgboost-diabetes-detection"
model_card_content = {
    "model_overview": {
                        "model_name": "XGBoost Diabetes Detection",
                        "model_description": "This model predicts the likelihood of diabetes based on health indicators using the XGBoost algorithm.",
                        "model_owner": "Massimiliano Repupilli",
                        "problem_type": "Binary Classification"
                      },
    "evaluation_details": [
                            {
                            "name": "Evaluation 1",
            
                            }
    ]
}

# Convert the Content dictionary to a JSON string
model_card_content_json = json.dumps(model_card_content)

# Create the Model Card
create_model_card_response = sagemaker_client.create_model_card(
    ModelCardName=model_card_name,
    Content=model_card_content_json,
    ModelCardStatus="Draft"  # Required parameter
)

print("Model Card ARN:", create_model_card_response['ModelCardArn'])

# Describe the Model Card
describe_model_card_response = sagemaker_client.describe_model_card(
    ModelCardName=model_card_name
)

print(describe_model_card_response)

Model Card ARN: arn:aws:sagemaker:us-east-1:333907264027:model-card/xgboost-diabetes-detection
{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:333907264027:model-card/xgboost-diabetes-detection', 'ModelCardName': 'xgboost-diabetes-detection', 'ModelCardVersion': 1, 'Content': '{"model_overview": {"model_name": "XGBoost Diabetes Detection", "model_description": "This model predicts the likelihood of diabetes based on health indicators using the XGBoost algorithm.", "model_owner": "Massimiliano Repupilli", "problem_type": "Binary Classification"}, "evaluation_details": [{"name": "Evaluation 1"}]}', 'ModelCardStatus': 'Draft', 'CreationTime': datetime.datetime(2024, 6, 4, 6, 7, 4, 540000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:333907264027:user-profile/d-zegxltkmyanc/maxrep', 'UserProfileName': 'maxrep', 'DomainId': 'd-zegxltkmyanc'}, 'LastModifiedTime': datetime.datetime(2024, 6, 4, 6, 7, 4, 540000, tzinfo=tzlocal()), 'LastModifiedBy': {'UserProfile

### - Update Existing Model Card

In [10]:
model_card_content_v2 = {
    "model_overview": {
        "model_name": "XGBoost Diabetes Detection",
        "model_description": "This model predicts the likelihood of diabetes based on health indicators using the XGBoost algorithm.",
        "model_owner": "Max Repupilli",
        "problem_type": "Binary Classification"
    },
    "evaluation_details": [
        {
            "name": "Evaluation 1"
        }
    ]
}

# Convert the Content dictionary to a JSON string
model_card_content_json = json.dumps(model_card_content_v2)

# Update the existing Model Card
update_model_card_response = sagemaker_client.update_model_card(
    ModelCardName=model_card_name,
    Content=model_card_content_json,
    ModelCardStatus="Draft"  # Required parameter
)

print("Updated Model Card ARN:", update_model_card_response['ModelCardArn'])

# Describe the Model Card
describe_model_card_response = sagemaker_client.describe_model_card(
    ModelCardName=model_card_name
)

print(describe_model_card_response)


Updated Model Card ARN: arn:aws:sagemaker:us-east-1:333907264027:model-card/xgboost-diabetes-detection
{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:333907264027:model-card/xgboost-diabetes-detection', 'ModelCardName': 'xgboost-diabetes-detection', 'ModelCardVersion': 2, 'Content': '{"model_overview": {"model_name": "XGBoost Diabetes Detection", "model_description": "This model predicts the likelihood of diabetes based on health indicators using the XGBoost algorithm.", "model_owner": "Max Repupilli", "problem_type": "Binary Classification"}, "evaluation_details": [{"name": "Evaluation 1"}]}', 'ModelCardStatus': 'Draft', 'CreationTime': datetime.datetime(2024, 6, 4, 6, 7, 4, 540000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:333907264027:user-profile/d-zegxltkmyanc/maxrep', 'UserProfileName': 'maxrep', 'DomainId': 'd-zegxltkmyanc'}, 'LastModifiedTime': datetime.datetime(2024, 6, 4, 6, 7, 10, 819000, tzinfo=tzlocal()), 'LastModifiedBy': {'UserProfile

### - Create a New Version of a Model Card

In [11]:
# Define a new Model Card name or version
new_model_card_name = "xgboost-diabetes-detection-v1"  # Or handle versioning within content

# Create the new Model Card
create_model_card_response = sagemaker_client.create_model_card(
    ModelCardName=new_model_card_name,
    Content=model_card_content_json,
    ModelCardStatus="Draft"  # Required parameter
)

print("New Model Card ARN:", create_model_card_response['ModelCardArn'])

# Describe the new Model Card
describe_model_card_response = sagemaker_client.describe_model_card(
    ModelCardName=new_model_card_name
)

print(describe_model_card_response)


New Model Card ARN: arn:aws:sagemaker:us-east-1:333907264027:model-card/xgboost-diabetes-detection-v1
{'ModelCardArn': 'arn:aws:sagemaker:us-east-1:333907264027:model-card/xgboost-diabetes-detection-v1', 'ModelCardName': 'xgboost-diabetes-detection-v1', 'ModelCardVersion': 1, 'Content': '{"model_overview": {"model_name": "XGBoost Diabetes Detection", "model_description": "This model predicts the likelihood of diabetes based on health indicators using the XGBoost algorithm.", "model_owner": "Max Repupilli", "problem_type": "Binary Classification"}, "evaluation_details": [{"name": "Evaluation 1"}]}', 'ModelCardStatus': 'Draft', 'CreationTime': datetime.datetime(2024, 6, 4, 6, 7, 13, 527000, tzinfo=tzlocal()), 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:333907264027:user-profile/d-zegxltkmyanc/maxrep', 'UserProfileName': 'maxrep', 'DomainId': 'd-zegxltkmyanc'}, 'LastModifiedTime': datetime.datetime(2024, 6, 4, 6, 7, 13, 527000, tzinfo=tzlocal()), 'LastModifiedBy': {'UserP

### - List ALL 'Existing' Model Cards

In [12]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all Model Cards
list_model_cards_response = sagemaker_client.list_model_cards()

# Print the list of Model Cards
for model_card in list_model_cards_response.get('ModelCardSummaries', []):
    print(f"Model Card Name: {model_card['ModelCardName']}, Creation Time: {model_card['CreationTime']}")

# If there are additional attributes, print them
for model_card in list_model_cards_response.get('ModelCardSummaries', []):
    for key, value in model_card.items():
        print(f"{key}: {value}")
    print("----")
if len(list_model_cards_response.get('ModelCardSummaries', [])) == 0:
    print('No Model Cards Exist')

Model Card Name: xgboost-diabetes-detection, Creation Time: 2024-06-04 06:07:04.540000+00:00
Model Card Name: xgboost-diabetes-detection-v1, Creation Time: 2024-06-04 06:07:13.527000+00:00
ModelCardName: xgboost-diabetes-detection
ModelCardArn: arn:aws:sagemaker:us-east-1:333907264027:model-card/xgboost-diabetes-detection
ModelCardStatus: Draft
CreationTime: 2024-06-04 06:07:04.540000+00:00
LastModifiedTime: 2024-06-04 06:07:10.819000+00:00
----
ModelCardName: xgboost-diabetes-detection-v1
ModelCardArn: arn:aws:sagemaker:us-east-1:333907264027:model-card/xgboost-diabetes-detection-v1
ModelCardStatus: Draft
CreationTime: 2024-06-04 06:07:13.527000+00:00
LastModifiedTime: 2024-06-04 06:07:13.527000+00:00
----


### - Delete ALL Cards in the List

In [13]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# List all Model Cards
list_model_cards_response = sagemaker_client.list_model_cards()

# Loop through each model card and delete it
for model_card in list_model_cards_response.get('ModelCardSummaries', []):
    model_card_name = model_card['ModelCardName']
    # Describe the model card to get the version
    describe_model_card_response = sagemaker_client.describe_model_card(
        ModelCardName=model_card_name,
        ModelCardVersion=1  # assuming version 1, update if needed
    )
    model_card_arn = describe_model_card_response['ModelCardArn']
    
    # Delete the model card
    print(f"Deleting Model Card: {model_card_name}")
    sagemaker_client.delete_model_card(ModelCardName=model_card_name)

print("All Model Cards have been deleted.")


Deleting Model Card: xgboost-diabetes-detection
Deleting Model Card: xgboost-diabetes-detection-v1
All Model Cards have been deleted.


### - Delete a Specific Card

In [ ]:
import boto3

# Initialize the SageMaker client
sagemaker_client = boto3.client('sagemaker')

# Specify the Model Card name you want to delete
model_card_name = "xgboost-diabetes-detection"

# Delete the Model Card
delete_model_card_response = sagemaker_client.delete_model_card(
    ModelCardName=model_card_name
)

print("Model Card deleted:", delete_model_card_response)